<a href="https://colab.research.google.com/github/SajalMShrestha/movie-recommender/blob/main/FineTuneChatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#INSTRUCTIONS: This runs an installation of OpenAI's libraries
!pip install openai

In [ ]:
# This code tells the compiler which code packages to use, and sets your API key
# Replace ADD YOUR OPENAI API KEY HERE with your own API Key from OpenAI
import os
import openai

openai.api_key = "ADD YOUR OPENAI API KEY HERE"

In [ ]:

# INSTRUCTIONS: upload your training data file to the 'content' directory to the left <-
# Replace 'yourFile.jsonl' with the name of your .jsonl file (downloaded from the lab instruction page at https://www.aicareerboost.com/products/the-complete-ai-product-leader-blueprint-2024/categories/2154368192/posts/2173638436)
# The code below will upload your training file to OpenAI's servers and return a file identified (called fileID).
# You only need to do this once per training file - once it is uploaded, it is on OpenAI's servers!

openai.File.create(
  file=open("yourFile.jsonl", "rb"),
  purpose='fine-tune'
)

In [ ]:
# INSTRUCTIONS: Replace 'training_file_id' with the "id" from the step above
openai.FineTuningJob.create(training_file="file-ro5dvFNCmho2czF51kLE1faJ", model="gpt-3.5-turbo")

# Great, you have kicked off the fine-tuning process on OpenAI's servers!

In [ ]:
#You can check on the state of your fine-tuned model by replacing 'Fine Tune Job Id' with the "id" from the command above
#Once you see a message telling you the fine-tuning is complete, it will also tell you the name of your NEW model!
openai.FineTuningJob.retrieve("ftjob-XzrpCEnYrVTOWuBBhnux2zJH")


In [7]:
# Install necessary library
!pip install tmdbv3api

# Import tools
from tmdbv3api import TMDb, Movie
tmdb = TMDb()

# Enter your TMDb API key here
tmdb.api_key = '1ab8629cf19578c0576135e9bd71bb23'

movie = Movie()

# Function: Recommend similar movies
def recommend_movie(movie_title):
    search_result = movie.search(movie_title)
    if not search_result:
        return "Movie not found"

    movie_id = search_result[0].id
    similar = movie.similar(movie_id)

    print(f"Because you liked '{movie_title}', you may also like:")
    for m in list(similar)[:5]:
        print("-", m.title)

# Try it with one of your favorite movies
recommend_movie("The Bourne Identity")


Because you liked 'The Bourne Identity', you may also like:
- The Dark Knight
- Edward Scissorhands
- Ocean's Twelve
- The Party
- The Party 2


In [8]:
# Install necessary library
!pip install tmdbv3api

# Import tools
from tmdbv3api import TMDb, Movie
from collections import Counter

tmdb = TMDb()

# Enter your TMDb API key here
tmdb.api_key = '1ab8629cf19578c0576135e9bd71bb23'

movie = Movie()

# Function: Recommend similar movies based on multiple favorites
def recommend_movies(favorite_titles):
    all_recommendations = []

    for title in favorite_titles:
        search_result = movie.search(title)
        if not search_result:
            print(f"Movie '{title}' not found. Skipping.")
            continue

        movie_id = search_result[0].id
        similar = movie.similar(movie_id)
        all_recommendations.extend([m.title for m in list(similar)])

    # Count and sort recommendations by frequency
    counter = Counter(all_recommendations)
    top_recommendations = counter.most_common(10)

    print("\nTop 10 Recommendations Based on Your Favorites:")
    for title, count in top_recommendations:
        print(f"- {title} (recommended {count} times)")

# Enter up to 5 of your favorite movies
favorite_movies = [
    "The Bourne Identity",
    "Knocked Up",
    "Manchester by the Sea",
    "Gone Girl",
    "Miami Vice"
]

recommend_movies(favorite_movies)



Top 10 Recommendations Based on Your Favorites:
- Blown Away (recommended 2 times)
- Lucky Number Slevin (recommended 2 times)
- The Terminator (recommended 2 times)
- Twilight (recommended 2 times)
- He's Just Not That Into You (recommended 2 times)
- The Dark Knight (recommended 1 times)
- Edward Scissorhands (recommended 1 times)
- Ocean's Twelve (recommended 1 times)
- The Party (recommended 1 times)
- The Party 2 (recommended 1 times)


In [10]:
# Install necessary library
!pip install tmdbv3api

# Import tools
from tmdbv3api import TMDb, Movie, Person
from collections import defaultdict
from datetime import datetime

# Setup TMDb API
tmdb = TMDb()
tmdb.api_key = '1ab8629cf19578c0576135e9bd71bb23'

movie = Movie()
person = Person()

# Function: Compute recommendation score
def compute_score(candidate, favorite_genres, favorite_actors):
    score = 0

    # Genre Match (25%)
    genre_match = len(set([g['name'] for g in candidate.genres]) & favorite_genres) / len(favorite_genres or [1])
    score += 25 * genre_match

    # Actor Match (20%)
    actor_match = len(set([a.name for a in list(candidate.cast)[:3]]) & favorite_actors) / len(favorite_actors or [1])
    score += 20 * actor_match

    # Year of Release
    year = int(candidate.release_date[:4]) if candidate.release_date else 0
    current_year = datetime.now().year
    if current_year - year <= 2:
        score += 15
    elif current_year - year <= 10:
        score += 10
    else:
        score += 5

    # IMDB/average vote score (25%)
    score += 25 * (candidate.vote_average or 0) / 10

    return score

# Function: Recommend similar movies based on weighted scoring
def recommend_movies(favorite_titles):
    favorite_genres = set()
    favorite_actors = set()
    candidate_movies = {}

    for title in favorite_titles:
        search_result = movie.search(title)
        if not search_result:
            print(f"Movie '{title}' not found. Skipping.")
            continue

        movie_details = movie.details(search_result[0].id)
        movie_credits = movie.credits(search_result[0].id)

        # Collect genres and top 3 cast members
        favorite_genres.update([g['name'] for g in movie_details.genres])
        favorite_actors.update([c['name'] for c in movie_credits['cast'][:3]])

        # Fetch similar movies
        similar = movie.similar(movie_details.id)
        for m in list(similar):
            if m.id not in candidate_movies:
                try:
                    m_details = movie.details(m.id)
                    m_credits = movie.credits(m.id)
                    m.genres = m_details.genres
                    m.cast = m_credits['cast'][:3]  # top 3 actors
                    candidate_movies[m.id] = m
                except:
                    continue

    # Score each candidate movie
    scored = [(m.title, compute_score(m, favorite_genres, favorite_actors)) for m in candidate_movies.values()]
    top_scored = sorted(scored, key=lambda x: x[1], reverse=True)[:10]

    print("\nTop 10 Recommendations Based on Your Favorites:")
    for title, score in top_scored:
        print(f"- {title} (score: {round(score, 2)})")

# Enter up to 5 of your favorite movies
favorite_movies = [
    "The Bourne Identity",
    "Knocked Up",
    "Manchester by the Sea",
    "Gone Girl",
    "Miami Vice"
]

recommend_movies(favorite_movies)


TypeError: attribute name must be string, not 'slice'

In [11]:
# Install necessary library
!pip install tmdbv3api

# Import tools
from tmdbv3api import TMDb, Movie, Person
from collections import defaultdict
from datetime import datetime

# Setup TMDb API
tmdb = TMDb()
tmdb.api_key = '1ab8629cf19578c0576135e9bd71bb23'

movie = Movie()
person = Person()

# Function: Compute recommendation score
def compute_score(candidate, favorite_genres, favorite_actors):
    score = 0

    # Genre Match (25%)
    genre_match = len(set([g['name'] for g in candidate.genres]) & favorite_genres) / len(favorite_genres or [1])
    score += 25 * genre_match

    # Actor Match (20%)
    actor_match = len(set([a.name for a in list(candidate.cast)[:3]]) & favorite_actors) / len(favorite_actors or [1])
    score += 20 * actor_match

    # Year of Release
    year = int(candidate.release_date[:4]) if candidate.release_date else 0
    current_year = datetime.now().year
    if current_year - year <= 2:
        score += 15
    elif current_year - year <= 10:
        score += 10
    else:
        score += 5

    # IMDB/average vote score (25%)
    score += 25 * (candidate.vote_average or 0) / 10

    return score

# Function: Recommend similar movies based on weighted scoring
def recommend_movies(favorite_titles):
    favorite_genres = set()
    favorite_actors = set()
    candidate_movies = {}

    for title in favorite_titles:
        search_result = movie.search(title)
        if not search_result:
            print(f"Movie '{title}' not found. Skipping.")
            continue

        movie_details = movie.details(search_result[0].id)
        movie_credits = movie.credits(search_result[0].id)

        # Collect genres and top 3 cast members
        favorite_genres.update([g['name'] for g in movie_details.genres])
        favorite_actors.update([c['name'] for c in list(movie_credits['cast'])[:3]])

        # Fetch similar movies
        similar = movie.similar(movie_details.id)
        for m in list(similar):
            if m.id not in candidate_movies:
                try:
                    m_details = movie.details(m.id)
                    m_credits = movie.credits(m.id)
                    m.genres = m_details.genres
                    m.cast = list(m_credits['cast'])[:3]  # top 3 actors
                    candidate_movies[m.id] = m
                except:
                    continue

    # Score each candidate movie
    scored = [(m.title, compute_score(m, favorite_genres, favorite_actors)) for m in candidate_movies.values()]
    top_scored = sorted(scored, key=lambda x: x[1], reverse=True)[:10]

    print("\nTop 10 Recommendations Based on Your Favorites:")
    for title, score in top_scored:
        print(f"- {title} (score: {round(score, 2)})")

# Enter up to 5 of your favorite movies
favorite_movies = [
    "The Bourne Identity",
    "Knocked Up",
    "Manchester by the Sea",
    "Gone Girl",
    "Miami Vice"
]

recommend_movies(favorite_movies)



Top 10 Recommendations Based on Your Favorites:
- Castle of Sand (score: 42.46)
- The Dark Knight (score: 40.59)
- Teenage Mutant Ninja Turtles: Mutant Mayhem (score: 40.2)
- Lucky Number Slevin (score: 38.0)
- Some Like It Hot (score: 35.95)
- Conspiracy Theory (score: 35.8)
- The Last Supper (score: 35.7)
- Dragonfly (score: 35.63)
- Rebecca (score: 35.46)
- The Name of the Rose (score: 34.45)


In [12]:
# Install necessary library
!pip install tmdbv3api

# Import tools
from tmdbv3api import TMDb, Movie, Person
from collections import defaultdict
from datetime import datetime
import concurrent.futures
import time

# Setup TMDb API
tmdb = TMDb()
tmdb.api_key = '1ab8629cf19578c0576135e9bd71bb23'

movie = Movie()
person = Person()

# Function: Compute recommendation score
def compute_score(candidate, favorite_genres, favorite_actors):
    score = 0

    # Genre Match (25%)
    genre_match = len(set([g['name'] for g in candidate.genres]) & favorite_genres) / len(favorite_genres or [1])
    score += 25 * genre_match

    # Actor Match (20%)
    actor_match = len(set([a.name for a in list(candidate.cast)[:3]]) & favorite_actors) / len(favorite_actors or [1])
    score += 20 * actor_match

    # Year of Release
    year = int(candidate.release_date[:4]) if candidate.release_date else 0
    current_year = datetime.now().year
    if current_year - year <= 2:
        score += 15
    elif current_year - year <= 10:
        score += 10
    else:
        score += 5

    # IMDB/average vote score (25%)
    score += 25 * (candidate.vote_average or 0) / 10

    return score

# Function to safely fetch and enrich similar movie data with basic logging
cache = {}
def fetch_similar_movie_details(m_id):
    if m_id in cache:
        return m_id, cache[m_id]
    try:
        m_details = movie.details(m_id)
        m_credits = movie.credits(m_id)
        m_details.genres = m_details.genres
        m_details.cast = list(m_credits['cast'])[:3]
        cache[m_id] = m_details
        print(f"Fetched: {m_details.title}")
        return m_id, m_details
    except Exception as e:
        print(f"Failed to fetch movie ID {m_id}: {e}")
        return m_id, None

# Function: Recommend similar movies based on weighted scoring
def recommend_movies(favorite_titles):
    start_time = time.time()
    favorite_genres = set()
    favorite_actors = set()
    candidate_movie_ids = set()

    for title in favorite_titles:
        search_result = movie.search(title)
        if not search_result:
            print(f"Movie '{title}' not found. Skipping.")
            continue

        movie_details = movie.details(search_result[0].id)
        movie_credits = movie.credits(search_result[0].id)

        # Collect genres and top 3 cast members
        favorite_genres.update([g['name'] for g in movie_details.genres])
        favorite_actors.update([c['name'] for c in list(movie_credits['cast'])[:3]])

        # Collect similar movie IDs
        similar = movie.similar(movie_details.id)
        candidate_movie_ids.update([m.id for m in list(similar)])

    # Use concurrent futures to fetch movie details in parallel
    candidate_movies = {}
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        future_to_id = {executor.submit(fetch_similar_movie_details, m_id): m_id for m_id in candidate_movie_ids}
        for future in concurrent.futures.as_completed(future_to_id):
            m_id, m = future.result()
            if m:
                candidate_movies[m_id] = m

    # Score each candidate movie
    scored = [(m.title, compute_score(m, favorite_genres, favorite_actors)) for m in candidate_movies.values()]
    top_scored = sorted(scored, key=lambda x: x[1], reverse=True)[:10]

    print("\nTop 10 Recommendations Based on Your Favorites:")
    for title, score in top_scored:
        print(f"- {title} (score: {round(score, 2)})")

    print(f"\nTotal execution time: {round(time.time() - start_time, 2)} seconds")

# Enter up to 5 of your favorite movies
favorite_movies = [
    "The Bourne Identity",
    "Knocked Up",
    "Manchester by the Sea",
    "Gone Girl",
    "Miami Vice"
]

recommend_movies(favorite_movies)


Fetched: Where the Truth Lies
Fetched: Castle of Sand
Fetched: Mutiny on the Bounty
Fetched: Joe Somebody
Fetched: Teenage Mutant Ninja Turtles: Mutant Mayhem
Fetched: Fifteen Year Old Captain
Fetched: Mad Max 2
Fetched: Genova
Fetched: Doom
Fetched: Conspiracy Theory
Fetched: The Taking of Pelham One Two Three
Fetched: A Perfect Day
Fetched: The Dark Knight
Fetched: Edward Scissorhands
Fetched: Ocean's Twelve
Fetched: The Party
Fetched: Star Trek V: The Final Frontier
Fetched: Blown Away
Fetched: Ocean's Eleven
Fetched: The Interpreter
Fetched: By the Gun
Fetched: The Son's Room
Fetched: Lucky Number Slevin
Fetched: Saw
Fetched: Jackie Brown
Fetched: The Party 2
Fetched: Minority Report
Fetched: Skin Deep
Fetched: Today You Die
Fetched: Life as a House
Fetched: The Name of the Rose
Fetched: The Wages of Fear
Fetched: Paradise Found
Fetched: Star Trek: Generations
Fetched: North by Northwest
Fetched: Little Indian, Big City
Fetched: A Man Apart
Fetched: The Hitcher
Fetched: Rebecca
Fet

In [13]:
# Install necessary library
!pip install tmdbv3api

# Import tools
from tmdbv3api import TMDb, Movie, Person
from collections import defaultdict
from datetime import datetime
import concurrent.futures
import time

# Setup TMDb API
tmdb = TMDb()
tmdb.api_key = '1ab8629cf19578c0576135e9bd71bb23'

movie = Movie()
person = Person()

# Recommendation weights and platform priorities
recommendation_weights = {
    "genre_similarity": 0.20,
    "cast_crew": 0.25,
    "release_year": 0.15,
    "ratings": 0.20,
    "streaming_availability": 0.10,
    "mood_tone": 0.05,
    "trending_factor": 0.05
}

streaming_platform_priority = {
    "netflix": 1.0,
    "disney_plus": 0.9,
    "hbo_max": 0.85,
    "hulu": 0.8,
    "prime_video": 0.75,
    "apple_tv": 0.7,
    "peacock": 0.6,
    "paramount_plus": 0.5,
}

# Function: Compute recommendation score
def compute_score(candidate, favorite_genres, favorite_actors, user_preferences):
    score = 0

    # Genre match (primary and secondary)
    genre_match = len(set([g['name'] for g in candidate.genres]) & favorite_genres) / len(favorite_genres or [1])
    score += recommendation_weights["genre_similarity"] * genre_match

    # Cast/Crew match
    actor_match = len(set([a.name for a in list(candidate.cast)[:3]]) & favorite_actors) / len(favorite_actors or [1])
    score += recommendation_weights["cast_crew"] * actor_match

    # Year of Release
    year = int(candidate.release_date[:4]) if candidate.release_date else 0
    current_year = datetime.now().year
    if current_year - year <= 2:
        score += recommendation_weights["release_year"] * 0.53
    elif current_year - year <= 5:
        score += recommendation_weights["release_year"] * 0.33
    elif current_year - year <= 15:
        score += recommendation_weights["release_year"] * 0.13

    # Ratings (IMDB proxy only for now)
    score += recommendation_weights["ratings"] * (candidate.vote_average or 0) / 10

    # Streaming availability boost
    platform = getattr(candidate, 'platform', None)
    if platform in user_preferences["subscribed_platforms"]:
        boost = streaming_platform_priority.get(platform, 0.5)
        score += recommendation_weights["streaming_availability"] * boost

    # Mood and trending left for future implementation
    return score

# Function to safely fetch and enrich similar movie data with basic logging
cache = {}
def fetch_similar_movie_details(m_id):
    if m_id in cache:
        return m_id, cache[m_id]
    try:
        m_details = movie.details(m_id)
        m_credits = movie.credits(m_id)
        m_details.genres = m_details.genres
        m_details.cast = list(m_credits['cast'])[:3]
        m_details.platform = None  # Placeholder
        cache[m_id] = m_details
        print(f"Fetched: {m_details.title}")
        return m_id, m_details
    except Exception as e:
        print(f"Failed to fetch movie ID {m_id}: {e}")
        return m_id, None

# Function: Recommend similar movies based on weighted scoring
def recommend_movies(favorite_titles):
    start_time = time.time()
    favorite_genres = set()
    favorite_actors = set()
    candidate_movie_ids = set()

    user_preferences = {
        "subscribed_platforms": ["netflix", "hbo_max", "prime_video"]
    }

    for title in favorite_titles:
        search_result = movie.search(title)
        if not search_result:
            print(f"Movie '{title}' not found. Skipping.")
            continue

        movie_details = movie.details(search_result[0].id)
        movie_credits = movie.credits(search_result[0].id)

        # Collect genres and top 3 cast members
        favorite_genres.update([g['name'] for g in movie_details.genres])
        favorite_actors.update([c['name'] for c in list(movie_credits['cast'])[:3]])

        # Collect similar movie IDs
        similar = movie.similar(movie_details.id)
        candidate_movie_ids.update([m.id for m in list(similar)])

    # Use concurrent futures to fetch movie details in parallel
    candidate_movies = {}
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        future_to_id = {executor.submit(fetch_similar_movie_details, m_id): m_id for m_id in candidate_movie_ids}
        for future in concurrent.futures.as_completed(future_to_id):
            m_id, m = future.result()
            if m:
                candidate_movies[m_id] = m

    # Score each candidate movie
    scored = [(m.title, compute_score(m, favorite_genres, favorite_actors, user_preferences)) for m in candidate_movies.values()]
    top_scored = sorted(scored, key=lambda x: x[1], reverse=True)[:10]

    print("\nTop 10 Recommendations Based on Your Favorites:")
    for title, score in top_scored:
        print(f"- {title} (score: {round(score, 2)})")

    print(f"\nTotal execution time: {round(time.time() - start_time, 2)} seconds")

# Enter up to 5 of your favorite movies
favorite_movies = [
    "The Bourne Identity",
    "Knocked Up",
    "Manchester by the Sea",
    "Gone Girl",
    "Miami Vice"
]

recommend_movies(favorite_movies)



Fetched: Where the Truth Lies
Fetched: Castle of Sand
Fetched: Teenage Mutant Ninja Turtles: Mutant Mayhem
Fetched: Mutiny on the Bounty
Fetched: Joe Somebody
Fetched: Fifteen Year Old Captain
Fetched: Mad Max 2
Fetched: Genova
Fetched: Conspiracy Theory
Fetched: Doom
Fetched: The Taking of Pelham One Two Three
Fetched: A Perfect Day
Fetched: Edward Scissorhands
Fetched: The Dark Knight
Fetched: Ocean's Eleven
Fetched: The Party
Fetched: The Party 2
Fetched: Saw
Fetched: Blown Away
Fetched: Star Trek V: The Final Frontier
Fetched: By the Gun
Fetched: The Son's Room
Fetched: Jackie Brown
Fetched: Ocean's Twelve
Fetched: Skin Deep
Fetched: Today You Die
Fetched: The Interpreter
Fetched: Life as a House
Fetched: The Name of the Rose
Fetched: Lucky Number Slevin
Fetched: Minority Report
Fetched: Star Trek: Generations
Fetched: The Wages of Fear
Fetched: Paradise Found
Fetched: The Hitcher
Fetched: Little Indian, Big City
Fetched: A Man Apart
Fetched: North by Northwest
Fetched: East of Ede

In [14]:
# Install necessary library
!pip install tmdbv3api

# Import tools
from tmdbv3api import TMDb, Movie, Person
from collections import defaultdict
from datetime import datetime
import concurrent.futures
import time

# Setup TMDb API
tmdb = TMDb()
tmdb.api_key = '1ab8629cf19578c0576135e9bd71bb23'

movie = Movie()
person = Person()

# Recommendation weights and platform priorities
recommendation_weights = {
    "genre_similarity": 0.15,
    "cast_crew": 0.30,
    "release_year": 0.10,
    "ratings": 0.25,
    "streaming_availability": 0.10,
    "mood_tone": 0.05,
    "trending_factor": 0.05
}

streaming_platform_priority = {
    "netflix": 1.0,
    "disney_plus": 0.9,
    "hbo_max": 0.85,
    "hulu": 0.8,
    "prime_video": 0.75,
    "apple_tv": 0.7,
    "peacock": 0.6,
    "paramount_plus": 0.5,
}

# Function: Compute recommendation score
def compute_score(candidate, favorite_genres, favorite_actors, user_preferences):
    score = 0
    base_similarity_boost = 0.2
    score += base_similarity_boost

    # Genre match (primary and secondary)
    genre_match = len(set([g['name'] for g in candidate.genres]) & favorite_genres) / len(favorite_genres or [1])
    score += recommendation_weights["genre_similarity"] * genre_match

    # Cast/Crew match
    actor_match = len(set([a.name for a in list(candidate.cast)[:3]]) & favorite_actors) / len(favorite_actors or [1])
    score += recommendation_weights["cast_crew"] * actor_match

    # Year of Release (rescaled to 0-1)
    year = int(candidate.release_date[:4]) if candidate.release_date else 0
    current_year = datetime.now().year
    if current_year - year <= 2:
        score += recommendation_weights["release_year"] * 1.0
    elif current_year - year <= 5:
        score += recommendation_weights["release_year"] * 0.66
    elif current_year - year <= 15:
        score += recommendation_weights["release_year"] * 0.33

    # Ratings (IMDB proxy only for now)
    score += recommendation_weights["ratings"] * (candidate.vote_average or 0) / 10

    # Streaming availability boost
    platform = getattr(candidate, 'platform', None)
    if platform in user_preferences["subscribed_platforms"]:
        boost = streaming_platform_priority.get(platform, 0.5)
        score += recommendation_weights["streaming_availability"] * boost

    # Mood and trending left for future implementation
    return score

# Function to safely fetch and enrich similar movie data with basic logging
cache = {}
def fetch_similar_movie_details(m_id):
    if m_id in cache:
        return m_id, cache[m_id]
    try:
        m_details = movie.details(m_id)
        m_credits = movie.credits(m_id)
        m_details.genres = m_details.genres
        m_details.cast = list(m_credits['cast'])[:3]
        m_details.platform = None  # Placeholder
        cache[m_id] = m_details
        print(f"Fetched: {m_details.title}")
        return m_id, m_details
    except Exception as e:
        print(f"Failed to fetch movie ID {m_id}: {e}")
        return m_id, None

# Function: Recommend similar movies based on weighted scoring
def recommend_movies(favorite_titles):
    start_time = time.time()
    favorite_genres = set()
    favorite_actors = set()
    candidate_movie_ids = set()

    user_preferences = {
        "subscribed_platforms": ["netflix", "hbo_max", "prime_video"]
    }

    for title in favorite_titles:
        search_result = movie.search(title)
        if not search_result:
            print(f"Movie '{title}' not found. Skipping.")
            continue

        movie_details = movie.details(search_result[0].id)
        movie_credits = movie.credits(search_result[0].id)

        # Collect genres and top 3 cast members
        favorite_genres.update([g['name'] for g in movie_details.genres])
        favorite_actors.update([c['name'] for c in list(movie_credits['cast'])[:3]])

        # Collect similar movie IDs
        similar = movie.similar(movie_details.id)
        candidate_movie_ids.update([m.id for m in list(similar)])

    # Use concurrent futures to fetch movie details in parallel
    candidate_movies = {}
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        future_to_id = {executor.submit(fetch_similar_movie_details, m_id): m_id for m_id in candidate_movie_ids}
        for future in concurrent.futures.as_completed(future_to_id):
            m_id, m = future.result()
            if m:
                candidate_movies[m_id] = m

    # Score each candidate movie
    scored = [(m.title, compute_score(m, favorite_genres, favorite_actors, user_preferences)) for m in candidate_movies.values()]
    top_scored = sorted(scored, key=lambda x: x[1], reverse=True)[:10]

    print("\nTop 10 Recommendations Based on Your Favorites:")
    for title, score in top_scored:
        print(f"- {title} (score: {round(score, 2)})")

    print(f"\nTotal execution time: {round(time.time() - start_time, 2)} seconds")

# Enter up to 5 of your favorite movies
favorite_movies = [
    "The Bourne Identity",
    "Knocked Up",
    "Manchester by the Sea",
    "Gone Girl",
    "Miami Vice"
]

recommend_movies(favorite_movies)


Fetched: Where the Truth Lies
Fetched: Castle of Sand
Fetched: Mutiny on the Bounty
Fetched: Fifteen Year Old Captain
Fetched: Teenage Mutant Ninja Turtles: Mutant Mayhem
Fetched: Mad Max 2
Fetched: Joe Somebody
Fetched: Genova
Fetched: The Taking of Pelham One Two Three
Fetched: A Perfect Day
Fetched: Conspiracy Theory
Fetched: Doom
Fetched: The Party
Fetched: Star Trek V: The Final Frontier
Fetched: Edward Scissorhands
Fetched: Ocean's Twelve
Fetched: Blown Away
Fetched: Saw
Fetched: The Party 2
Fetched: By the Gun
Fetched: The Son's Room
Fetched: Ocean's Eleven
Fetched: The Interpreter
Fetched: Jackie Brown
Fetched: The Dark Knight
Fetched: Skin Deep
Fetched: Lucky Number Slevin
Fetched: Minority Report
Fetched: Life as a House
Fetched: Paradise Found
Fetched: The Hitcher
Fetched: Star Trek: Generations
Fetched: Little Indian, Big City
Fetched: A Man Apart
Fetched: The Wages of Fear
Fetched: Rebecca
Fetched: Beck 06 - The Monster
Fetched: Today You Die
Fetched: The Blue Angel
Fetche

In [15]:
# Install necessary library
!pip install tmdbv3api

# Import tools
from tmdbv3api import TMDb, Movie, Person
from collections import defaultdict
from datetime import datetime
import concurrent.futures
import time

# Setup TMDb API
tmdb = TMDb()
tmdb.api_key = '1ab8629cf19578c0576135e9bd71bb23'

movie = Movie()
person = Person()

# Updated recommendation weights and platform priorities
recommendation_weights = {
    "genre_similarity": 0.20,
    "cast_crew": 0.25,
    "release_year": 0.15,
    "ratings": 0.20,
    "streaming_availability": 0.10,
    "mood_tone": 0.05,
    "trending_factor": 0.05
}

streaming_platform_priority = {
    "netflix": 1.0,
    "disney_plus": 0.9,
    "hbo_max": 0.85,
    "hulu": 0.8,
    "prime_video": 0.75,
    "apple_tv": 0.7,
    "peacock": 0.6,
    "paramount_plus": 0.5,
}

mood_tone_map = {
    "feel_good": {"Comedy", "Romance", "Music", "Adventure"},
    "gritty": {"Crime", "Thriller", "Mystery", "Drama"},
    "cerebral": {"Sci-Fi", "Mystery", "History"},
    "intense": {"Action", "War", "Horror"},
    "melancholic": {"Drama", "History"},
    "classic": {"Western", "Film-Noir"}
}

# Maturity filter
immature_genres = {"Family", "Animation", "Kids"}

def get_maturity_penalty(genres):
    return 0.15 if any(g['name'] in immature_genres for g in genres) else 0

# Function: Compute recommendation score

def get_mood_score(genres, preferred_moods):
    matched_moods = set()
    for g in genres:
        for mood, tags in mood_tone_map.items():
            if g['name'] in tags:
                matched_moods.add(mood)
    overlap = matched_moods & preferred_moods
    return len(overlap) / len(preferred_moods or [1])

def compute_score(candidate, favorite_genres, favorite_actors, user_preferences):
    score = 0
    base_similarity_boost = 0.2
    score += base_similarity_boost

    # Genre match (primary and secondary)
    genre_match = len(set([g['name'] for g in candidate.genres]) & favorite_genres) / len(favorite_genres or [1])
    score += recommendation_weights["genre_similarity"] * genre_match

    # Cast/Crew match
    actor_match = len(set([a.name for a in list(candidate.cast)[:3]]) & favorite_actors) / len(favorite_actors or [1])
    score += recommendation_weights["cast_crew"] * actor_match

    # Year of Release (rescaled to 0-1)
    year = int(candidate.release_date[:4]) if candidate.release_date else 0
    current_year = datetime.now().year
    if current_year - year <= 2:
        score += recommendation_weights["release_year"] * 0.53
    elif current_year - year <= 5:
        score += recommendation_weights["release_year"] * 0.33
    elif current_year - year <= 15:
        score += recommendation_weights["release_year"] * 0.13

    # Ratings (IMDB proxy only for now)
    score += recommendation_weights["ratings"] * (candidate.vote_average or 0) / 10

    # Streaming availability boost
    platform = getattr(candidate, 'platform', None)
    if platform in user_preferences["subscribed_platforms"]:
        boost = streaming_platform_priority.get(platform, 0.5)
        score += recommendation_weights["streaming_availability"] * boost

    # Mood tone match
    mood_score = get_mood_score(candidate.genres, user_preferences.get("preferred_moods", set()))
    score += recommendation_weights["mood_tone"] * mood_score

    # Maturity level adjustment
    score -= get_maturity_penalty(candidate.genres)

    return score

# Function to safely fetch and enrich similar movie data with basic logging
cache = {}
def fetch_similar_movie_details(m_id):
    if m_id in cache:
        return m_id, cache[m_id]
    try:
        m_details = movie.details(m_id)
        m_credits = movie.credits(m_id)
        m_details.genres = m_details.genres
        m_details.cast = list(m_credits['cast'])[:3]
        m_details.platform = None  # Placeholder
        cache[m_id] = m_details
        print(f"Fetched: {m_details.title}")
        return m_id, m_details
    except Exception as e:
        print(f"Failed to fetch movie ID {m_id}: {e}")
        return m_id, None

# Function: Recommend similar movies based on weighted scoring
def recommend_movies(favorite_titles):
    start_time = time.time()
    favorite_genres = set()
    favorite_actors = set()
    candidate_movie_ids = set()

    user_preferences = {
        "subscribed_platforms": ["netflix", "hbo_max", "prime_video"],
        "preferred_moods": {"gritty", "melancholic"}
    }

    for title in favorite_titles:
        search_result = movie.search(title)
        if not search_result:
            print(f"Movie '{title}' not found. Skipping.")
            continue

        movie_details = movie.details(search_result[0].id)
        movie_credits = movie.credits(search_result[0].id)

        # Collect genres and top 3 cast members
        favorite_genres.update([g['name'] for g in movie_details.genres])
        favorite_actors.update([c['name'] for c in list(movie_credits['cast'])[:3]])

        # Collect similar movie IDs
        similar = movie.similar(movie_details.id)
        candidate_movie_ids.update([m.id for m in list(similar)])

    # Use concurrent futures to fetch movie details in parallel
    candidate_movies = {}
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        future_to_id = {executor.submit(fetch_similar_movie_details, m_id): m_id for m_id in candidate_movie_ids}
        for future in concurrent.futures.as_completed(future_to_id):
            m_id, m = future.result()
            if m:
                candidate_movies[m_id] = m

    # Score each candidate movie
    scored = [(m.title, compute_score(m, favorite_genres, favorite_actors, user_preferences)) for m in candidate_movies.values()]
    top_scored = sorted(scored, key=lambda x: x[1], reverse=True)[:10]

    print("\nTop 10 Recommendations Based on Your Favorites:")
    for title, score in top_scored:
        print(f"- {title} (score: {round(score, 2)})")

    print(f"\nTotal execution time: {round(time.time() - start_time, 2)} seconds")

# Enter up to 5 of your favorite movies
favorite_movies = [
    "The Bourne Identity",
    "Knocked Up",
    "Manchester by the Sea",
    "Gone Girl",
    "Miami Vice"
]

recommend_movies(favorite_movies)


Fetched: Where the Truth LiesFetched: Castle of Sand
Fetched: Teenage Mutant Ninja Turtles: Mutant Mayhem

Fetched: Joe Somebody
Fetched: Fifteen Year Old Captain
Fetched: Mutiny on the Bounty
Fetched: Mad Max 2
Fetched: Genova
Fetched: Conspiracy Theory
Fetched: Doom
Fetched: A Perfect Day
Fetched: The Taking of Pelham One Two Three
Fetched: The Dark Knight
Fetched: The Party
Fetched: Star Trek V: The Final Frontier
Fetched: Ocean's Eleven
Fetched: Edward Scissorhands
Fetched: Saw
Fetched: The Interpreter
Fetched: By the Gun
Fetched: Blown Away
Fetched: The Party 2
Fetched: Ocean's Twelve
Fetched: The Son's Room
Fetched: Minority Report
Fetched: Skin Deep
Fetched: Jackie Brown
Fetched: Today You Die
Fetched: The Name of the Rose
Fetched: Lucky Number Slevin
Fetched: The Wages of Fear
Fetched: Paradise Found
Fetched: North by Northwest
Fetched: Star Trek: Generations
Fetched: A Man Apart
Fetched: Little Indian, Big City
Fetched: Rebecca
Fetched: Life as a House
Fetched: The Outsiders
F

In [1]:
# Install necessary library
!pip install tmdbv3api

# Import tools
from tmdbv3api import TMDb, Movie, Person
from collections import defaultdict
from datetime import datetime
import concurrent.futures
import time

# Setup TMDb API
tmdb = TMDb()
tmdb.api_key = '1ab8629cf19578c0576135e9bd71bb23'

movie = Movie()
person = Person()

# Updated recommendation weights and platform priorities
recommendation_weights = {
    "genre_similarity": 0.20,
    "cast_crew": 0.25,
    "release_year": 0.15,
    "ratings": 0.20,
    "streaming_availability": 0.10,
    "mood_tone": 0.05,
    "trending_factor": 0.05
}

streaming_platform_priority = {
    "netflix": 1.0,
    "disney_plus": 0.9,
    "hbo_max": 0.85,
    "hulu": 0.8,
    "prime_video": 0.75,
    "apple_tv": 0.7,
    "peacock": 0.6,
    "paramount_plus": 0.5,
}

mood_tone_map = {
    "feel_good": {"Comedy", "Romance", "Music", "Adventure"},
    "gritty": {"Crime", "Thriller", "Mystery", "Drama"},
    "cerebral": {"Sci-Fi", "Mystery", "History"},
    "intense": {"Action", "War", "Horror"},
    "melancholic": {"Drama", "History"},
    "classic": {"Western", "Film-Noir"}
}

# Maturity filter
immature_genres = {"Family", "Animation", "Kids"}

def get_maturity_penalty(genres):
    return 0.15 if any(g['name'] in immature_genres for g in genres) else 0

# Function: Compute recommendation score

def get_mood_score(genres, preferred_moods):
    matched_moods = set()
    for g in genres:
        for mood, tags in mood_tone_map.items():
            if g['name'] in tags:
                matched_moods.add(mood)
    overlap = matched_moods & preferred_moods
    return len(overlap) / len(preferred_moods or [1])

def infer_moods_from_genres(genres):
    inferred = set()
    for g in genres:
        for mood, tags in mood_tone_map.items():
            if g in tags:
                inferred.add(mood)
    return inferred

def compute_score(candidate, favorite_genres, favorite_actors, user_preferences):
    score = 0
    base_similarity_boost = 0.2
    score += base_similarity_boost

    # Genre match (primary and secondary)
    genre_match = len(set([g['name'] for g in candidate.genres]) & favorite_genres) / len(favorite_genres or [1])
    score += recommendation_weights["genre_similarity"] * genre_match

    # Cast/Crew match
    actor_match = len(set([a.name for a in list(candidate.cast)[:3]]) & favorite_actors) / len(favorite_actors or [1])
    score += recommendation_weights["cast_crew"] * actor_match

    # Year of Release (rescaled to 0-1)
    year = int(candidate.release_date[:4]) if candidate.release_date else 0
    current_year = datetime.now().year
    if current_year - year <= 2:
        score += recommendation_weights["release_year"] * 0.53
    elif current_year - year <= 5:
        score += recommendation_weights["release_year"] * 0.33
    elif current_year - year <= 15:
        score += recommendation_weights["release_year"] * 0.13

    # Ratings (IMDB proxy only for now)
    score += recommendation_weights["ratings"] * (candidate.vote_average or 0) / 10

    # Streaming availability boost
    platform = getattr(candidate, 'platform', None)
    if platform in user_preferences["subscribed_platforms"]:
        boost = streaming_platform_priority.get(platform, 0.5)
        score += recommendation_weights["streaming_availability"] * boost

    # Mood tone match
    mood_score = get_mood_score(candidate.genres, user_preferences.get("preferred_moods", set()))
    score += recommendation_weights["mood_tone"] * mood_score

    # Maturity level adjustment
    score -= get_maturity_penalty(candidate.genres)

    return score

# Function to safely fetch and enrich similar movie data with basic logging
cache = {}
def fetch_similar_movie_details(m_id):
    if m_id in cache:
        return m_id, cache[m_id]
    try:
        m_details = movie.details(m_id)
        m_credits = movie.credits(m_id)
        m_details.genres = m_details.genres
        m_details.cast = list(m_credits['cast'])[:3]
        m_details.platform = None  # Placeholder
        cache[m_id] = m_details
        print(f"Fetched: {m_details.title}")
        return m_id, m_details
    except Exception as e:
        print(f"Failed to fetch movie ID {m_id}: {e}")
        return m_id, None

# Function: Recommend similar movies based on weighted scoring
def recommend_movies(favorite_titles):
    start_time = time.time()
    favorite_genres = set()
    favorite_actors = set()
    candidate_movie_ids = set()

    for title in favorite_titles:
        search_result = movie.search(title)
        if not search_result:
            print(f"Movie '{title}' not found. Skipping.")
            continue

        movie_details = movie.details(search_result[0].id)
        movie_credits = movie.credits(search_result[0].id)

        # Collect genres and top 3 cast members
        favorite_genres.update([g['name'] for g in movie_details.genres])
        favorite_actors.update([c['name'] for c in list(movie_credits['cast'])[:3]])

        # Collect similar movie IDs
        similar = movie.similar(movie_details.id)
        candidate_movie_ids.update([m.id for m in list(similar)])

    # Infer mood from favorite genres
    inferred_moods = infer_moods_from_genres(favorite_genres)
    user_preferences = {
        "subscribed_platforms": ["netflix", "hbo_max", "prime_video"],
        "preferred_moods": inferred_moods
    }

    # Use concurrent futures to fetch movie details in parallel
    candidate_movies = {}
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        future_to_id = {executor.submit(fetch_similar_movie_details, m_id): m_id for m_id in candidate_movie_ids}
        for future in concurrent.futures.as_completed(future_to_id):
            m_id, m = future.result()
            if m:
                candidate_movies[m_id] = m

    # Score each candidate movie
    scored = [(m.title, compute_score(m, favorite_genres, favorite_actors, user_preferences)) for m in candidate_movies.values()]
    top_scored = sorted(scored, key=lambda x: x[1], reverse=True)[:10]

    print("\nTop 10 Recommendations Based on Your Favorites:")
    for title, score in top_scored:
        print(f"- {title} (score: {round(score, 2)})")

    print(f"\nTotal execution time: {round(time.time() - start_time, 2)} seconds")

# Enter up to 5 of your favorite movies
favorite_movies = [
    "The Bourne Identity",
    "Knocked Up",
    "Manchester by the Sea",
    "Gone Girl",
    "Miami Vice"
]

recommend_movies(favorite_movies)



TMDbException: Invalid API key: You must be granted a valid key.

In [17]:
# Install necessary library
!pip install tmdbv3api
!pip install nltk

# Import tools
from tmdbv3api import TMDb, Movie, Person
from collections import defaultdict
from datetime import datetime
import concurrent.futures
import time
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

# Setup TMDb API
tmdb = TMDb()
tmdb.api_key = '1ab8629cf19578c0576135e9bd71bb23'

movie = Movie()
person = Person()
sia = SentimentIntensityAnalyzer()

# Updated recommendation weights and platform priorities
recommendation_weights = {
    "genre_similarity": 0.20,
    "cast_crew": 0.25,
    "release_year": 0.15,
    "ratings": 0.20,
    "streaming_availability": 0.10,
    "mood_tone": 0.05,
    "trending_factor": 0.05
}

streaming_platform_priority = {
    "netflix": 1.0,
    "disney_plus": 0.9,
    "hbo_max": 0.85,
    "hulu": 0.8,
    "prime_video": 0.75,
    "apple_tv": 0.7,
    "peacock": 0.6,
    "paramount_plus": 0.5,
}

mood_tone_map = {
    "feel_good": {"Comedy", "Romance", "Music", "Adventure"},
    "gritty": {"Crime", "Thriller", "Mystery", "Drama"},
    "cerebral": {"Sci-Fi", "Mystery", "History"},
    "intense": {"Action", "War", "Horror"},
    "melancholic": {"Drama", "History"},
    "classic": {"Western", "Film-Noir"}
}

# Maturity filter
immature_genres = {"Family", "Animation", "Kids"}

def get_maturity_penalty(genres):
    return 0.15 if any(g['name'] in immature_genres for g in genres) else 0

# Function: Compute recommendation score

def get_mood_score(genres, preferred_moods):
    matched_moods = set()
    for g in genres:
        for mood, tags in mood_tone_map.items():
            if g['name'] in tags:
                matched_moods.add(mood)
    overlap = matched_moods & preferred_moods
    return len(overlap) / len(preferred_moods or [1])

def infer_moods_from_genres(genres):
    inferred = set()
    for g in genres:
        for mood, tags in mood_tone_map.items():
            if g in tags:
                inferred.add(mood)
    return inferred

def infer_mood_from_plot(plot):
    sentiment = sia.polarity_scores(plot)
    if sentiment['compound'] >= 0.4:
        return 'feel_good'
    elif sentiment['compound'] <= -0.3:
        return 'melancholic'
    else:
        return 'cerebral'

def compute_score(candidate, favorite_genres, favorite_actors, user_preferences):
    score = 0
    base_similarity_boost = 0.2
    score += base_similarity_boost

    # Genre match (primary and secondary)
    genre_match = len(set([g['name'] for g in candidate.genres]) & favorite_genres) / len(favorite_genres or [1])
    score += recommendation_weights["genre_similarity"] * genre_match

    # Cast/Crew match
    actor_match = len(set([a.name for a in list(candidate.cast)[:3]]) & favorite_actors) / len(favorite_actors or [1])
    score += recommendation_weights["cast_crew"] * actor_match

    # Year of Release (rescaled to 0-1)
    year = int(candidate.release_date[:4]) if candidate.release_date else 0
    current_year = datetime.now().year
    if current_year - year <= 2:
        score += recommendation_weights["release_year"] * 0.53
    elif current_year - year <= 5:
        score += recommendation_weights["release_year"] * 0.33
    elif current_year - year <= 15:
        score += recommendation_weights["release_year"] * 0.13

    # Ratings (IMDB proxy only for now)
    score += recommendation_weights["ratings"] * (candidate.vote_average or 0) / 10

    # Streaming availability boost
    platform = getattr(candidate, 'platform', None)
    if platform in user_preferences["subscribed_platforms"]:
        boost = streaming_platform_priority.get(platform, 0.5)
        score += recommendation_weights["streaming_availability"] * boost

    # Mood tone match
    mood_score = get_mood_score(candidate.genres, user_preferences.get("preferred_moods", set()))
    score += recommendation_weights["mood_tone"] * mood_score

    # Maturity level adjustment
    score -= get_maturity_penalty(candidate.genres)

    return score

# Function to safely fetch and enrich similar movie data with basic logging
cache = {}
def fetch_similar_movie_details(m_id):
    if m_id in cache:
        return m_id, cache[m_id]
    try:
        m_details = movie.details(m_id)
        m_credits = movie.credits(m_id)
        m_details.genres = m_details.genres
        m_details.cast = list(m_credits['cast'])[:3]
        m_details.platform = None  # Placeholder
        m_details.plot = m_details.overview or ""
        cache[m_id] = m_details
        print(f"Fetched: {m_details.title}")
        return m_id, m_details
    except Exception as e:
        print(f"Failed to fetch movie ID {m_id}: {e}")
        return m_id, None

# Function: Recommend similar movies based on weighted scoring
def recommend_movies(favorite_titles):
    start_time = time.time()
    favorite_genres = set()
    favorite_actors = set()
    candidate_movie_ids = set()
    inferred_plot_moods = set()

    for title in favorite_titles:
        search_result = movie.search(title)
        if not search_result:
            print(f"Movie '{title}' not found. Skipping.")
            continue

        movie_details = movie.details(search_result[0].id)
        movie_credits = movie.credits(search_result[0].id)

        # Collect genres and top 3 cast members
        favorite_genres.update([g['name'] for g in movie_details.genres])
        favorite_actors.update([c['name'] for c in list(movie_credits['cast'])[:3]])

        # Infer mood from plot description
        inferred_plot_moods.add(infer_mood_from_plot(movie_details.overview or ""))

        # Collect similar movie IDs
        similar = movie.similar(movie_details.id)
        candidate_movie_ids.update([m.id for m in list(similar)])

    inferred_moods = infer_moods_from_genres(favorite_genres) | inferred_plot_moods
    user_preferences = {
        "subscribed_platforms": ["netflix", "hbo_max", "prime_video"],
        "preferred_moods": inferred_moods
    }

    # Use concurrent futures to fetch movie details in parallel
    candidate_movies = {}
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        future_to_id = {executor.submit(fetch_similar_movie_details, m_id): m_id for m_id in candidate_movie_ids}
        for future in concurrent.futures.as_completed(future_to_id):
            m_id, m = future.result()
            if m:
                candidate_movies[m_id] = m

    # Score each candidate movie
    scored = [(m.title, compute_score(m, favorite_genres, favorite_actors, user_preferences)) for m in candidate_movies.values()]
    top_scored = sorted(scored, key=lambda x: x[1], reverse=True)[:10]

    print("\nTop 10 Recommendations Based on Your Favorites:")
    for title, score in top_scored:
        print(f"- {title} (score: {round(score, 2)})")

    print(f"\nTotal execution time: {round(time.time() - start_time, 2)} seconds")

# Enter up to 5 of your favorite movies
favorite_movies = [
    "The Bourne Identity",
    "Knocked Up",
    "Manchester by the Sea",
    "Gone Girl",
    "Miami Vice"
]

recommend_movies(favorite_movies)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


Fetched: Where the Truth Lies
Fetched: Castle of Sand
Fetched: Teenage Mutant Ninja Turtles: Mutant Mayhem
Fetched: Mutiny on the Bounty
Fetched: Joe Somebody
Fetched: Fifteen Year Old Captain
Fetched: Mad Max 2
Fetched: Genova
Fetched: Doom
Fetched: The Taking of Pelham One Two Three
Fetched: A Perfect Day
Fetched: Conspiracy Theory
Fetched: Edward Scissorhands
Fetched: The Party
Fetched: Ocean's Eleven
Fetched: The Dark Knight
Fetched: The Party 2
Fetched: Ocean's Twelve
Fetched: Star Trek V: The Final Frontier
Fetched: Blown Away
Fetched: Saw
Fetched: The Interpreter
Fetched: The Son's Room
Fetched: By the Gun
Fetched: Lucky Number Slevin
Fetched: Skin Deep
Fetched: The Name of the Rose
Fetched: Jackie Brown
Fetched: Minority Report
Fetched: Star Trek: Generations
Fetched: The Wages of Fear
Fetched: Paradise Found
Fetched: North by Northwest
Fetched: A Man Apart
Fetched: Today You Die
Fetched: Little Indian, Big City
Fetched: The Hitcher
Fetched: Life as a House
Fetched: Rebecca
Fet

In [3]:
# Install necessary libraries
!pip install tmdbv3api nltk

# Import libraries
from tmdbv3api import TMDb, Movie, Person
from collections import defaultdict
from datetime import datetime
import concurrent.futures
import time
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

# TMDb setup
tmdb = TMDb()
tmdb.api_key = 'b2685c31a5f9519a6fa30472b7be2ca1'
tmdb.language = 'en'
tmdb.debug = True

movie = Movie()
person = Person()
sia = SentimentIntensityAnalyzer()

# Recommendation weights and streaming platform priorities
recommendation_weights = {
    "genre_similarity": 0.20,
    "cast_crew": 0.25,
    "release_year": 0.15,
    "ratings": 0.20,
    "streaming_availability": 0.10,
    "mood_tone": 0.05,
    "trending_factor": 0.05
}

streaming_platform_priority = {
    "netflix": 1.0,
    "disney_plus": 0.9,
    "hbo_max": 0.85,
    "hulu": 0.8,
    "prime_video": 0.75,
    "apple_tv": 0.7,
    "peacock": 0.6,
    "paramount_plus": 0.5,
}

mood_tone_map = {
    "feel_good": {"Comedy", "Romance", "Music", "Adventure"},
    "gritty": {"Crime", "Thriller", "Mystery", "Drama"},
    "cerebral": {"Sci-Fi", "Mystery", "History"},
    "intense": {"Action", "War", "Horror"},
    "melancholic": {"Drama", "History"},
    "classic": {"Western", "Film-Noir"}
}

def get_mood_score(genres, preferred_moods):
    matched_moods = set()
    for g in genres:
        for mood, tags in mood_tone_map.items():
            if g['name'] in tags:
                matched_moods.add(mood)
    overlap = matched_moods & preferred_moods
    return len(overlap) / len(preferred_moods or [1])

def infer_moods_from_genres(genres):
    inferred = set()
    for g in genres:
        for mood, tags in mood_tone_map.items():
            if g in tags:
                inferred.add(mood)
    return inferred

def infer_mood_from_plot(plot):
    sentiment = sia.polarity_scores(plot)
    if sentiment['compound'] >= 0.4:
        return 'feel_good'
    elif sentiment['compound'] <= -0.3:
        return 'melancholic'
    else:
        return 'cerebral'

def compute_score(candidate, favorite_genres, favorite_actors, user_preferences):
    score = 0
    base_similarity_boost = 0.2
    score += base_similarity_boost

    genre_match = len(set([g['name'] for g in candidate.genres]) & favorite_genres) / len(favorite_genres or [1])
    score += recommendation_weights["genre_similarity"] * genre_match

    actor_match = len(set([a.name for a in list(candidate.cast)[:3]]) & favorite_actors) / len(favorite_actors or [1])
    score += recommendation_weights["cast_crew"] * actor_match

    year = int(candidate.release_date[:4]) if candidate.release_date else 0
    current_year = datetime.now().year
    if current_year - year <= 2:
        score += recommendation_weights["release_year"] * 0.53
    elif current_year - year <= 5:
        score += recommendation_weights["release_year"] * 0.33
    elif current_year - year <= 15:
        score += recommendation_weights["release_year"] * 0.13

    score += recommendation_weights["ratings"] * (candidate.vote_average or 0) / 10

    platform = getattr(candidate, 'platform', None)
    if platform in user_preferences["subscribed_platforms"]:
        boost = streaming_platform_priority.get(platform, 0.5)
        score += recommendation_weights["streaming_availability"] * boost

    mood_score = get_mood_score(candidate.genres, user_preferences.get("preferred_moods", set()))
    score += recommendation_weights["mood_tone"] * mood_score

    return score

# Fetch similar movie details with caching
cache = {}
def fetch_similar_movie_details(m_id):
    if m_id in cache:
        return m_id, cache[m_id]
    try:
        m_details = movie.details(m_id)
        m_credits = movie.credits(m_id)
        m_details.genres = m_details.genres
        m_details.cast = list(m_credits['cast'])[:3]
        m_details.platform = None  # Placeholder
        m_details.plot = m_details.overview or ""
        cache[m_id] = m_details
        print(f"Fetched: {m_details.title}")
        return m_id, m_details
    except Exception as e:
        print(f"Failed to fetch movie ID {m_id}: {e}")
        return m_id, None

def recommend_movies(favorite_titles):
    start_time = time.time()
    favorite_genres = set()
    favorite_actors = set()
    candidate_movie_ids = set()
    inferred_plot_moods = set()

    for title in favorite_titles:
        search_result = movie.search(title)
        if not search_result:
            print(f"Movie '{title}' not found. Skipping.")
            continue

        movie_details = movie.details(search_result[0].id)
        movie_credits = movie.credits(search_result[0].id)

        favorite_genres.update([g['name'] for g in movie_details.genres])
        favorite_actors.update([c['name'] for c in list(movie_credits['cast'])[:3]])

        inferred_plot_moods.add(infer_mood_from_plot(movie_details.overview or ""))

        similar = movie.similar(movie_details.id)
        candidate_movie_ids.update([m.id for m in list(similar)])

    inferred_moods = infer_moods_from_genres(favorite_genres) | inferred_plot_moods
    user_preferences = {
        "subscribed_platforms": ["netflix", "hbo_max", "prime_video"],
        "preferred_moods": inferred_moods
    }

    candidate_movies = {}
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        future_to_id = {executor.submit(fetch_similar_movie_details, m_id): m_id for m_id in candidate_movie_ids}
        for future in concurrent.futures.as_completed(future_to_id):
            m_id, m = future.result()
            if m:
                candidate_movies[m_id] = m

    scored = [(m.title, compute_score(m, favorite_genres, favorite_actors, user_preferences)) for m in candidate_movies.values()]
    top_scored = sorted(scored, key=lambda x: x[1], reverse=True)[:10]

    print("\nTop 10 Recommendations Based on Your Favorites:")
    for title, score in top_scored:
        print(f"- {title} (score: {round(score, 2)})")

    print(f"\nTotal execution time: {round(time.time() - start_time, 2)} seconds")

# Example input
favorite_movies = [
    "The Bourne Identity",
    "Knocked Up",
    "Manchester by the Sea",
    "Gone Girl",
    "Miami Vice"
]

recommend_movies(favorite_movies)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Fetched: Emekli Başkan
Fetched: The Man from London
Fetched: Chloe
Fetched: Genome Hazard
Fetched: Ala Vaikunthapurramuloo
Fetched: So Cold the River
Fetched: Kabul Express
Fetched: La Vie de Bohème
Fetched: Sylvia
Fetched: Kandahar
Fetched: Birds Without Names
Fetched: Rock Steady
Fetched: Get a Clue
Fetched: The Nude Bomb
Fetched: The Secret Lives of Dentists
Fetched: The Maltese Falcon
Fetched: The Stone Killer
Fetched: Lured
Fetched: Flags of Our Fathers
Fetched: Kamen Rider Ex-Aid the Movie: True Ending
Fetched: Attack of the Robots
Fetched: Rosenbaum - Bländverk
Fetched: Superhero Movie
Fetched: Rosenbaum - Målbrott
Fetched: Last Train from Hiroshima
Fetched: Whisper Breach
Fetched: The Tin Drum
Fetched: Butch Cassidy and the Sundance Kid
Fetched: Cheese
Fetched: Orphan of Lowood
Fetched: Peut-être
Fetched: The Railway Station Man
Fetched: Promised a Miracle
Fetched: Rosenbaum - Det sista vittnet
Fetched: Tempting Fate
Fetched: Tower of London
Fetched: Maggie Marvel
Fetched: A Se

In [5]:
# Updated recommendation weights and platform priorities
recommendation_weights = {
    "genre_similarity": 0.20,
    "cast_crew": 0.25,
    "release_year": 0.15,
    "ratings": 0.20,
    "streaming_availability": 0.10,
    "mood_tone": 0.05,
    "trending_factor": 0.05
}

streaming_platform_priority = {
    "netflix": 1.0,
    "disney_plus": 0.9,
    "hbo_max": 0.85,
    "hulu": 0.8,
    "prime_video": 0.75,
    "apple_tv": 0.7,
    "peacock": 0.6,
    "paramount_plus": 0.5,
}

mood_tone_map = {
    "feel_good": {"Comedy", "Romance", "Music", "Adventure"},
    "gritty": {"Crime", "Thriller", "Mystery", "Drama"},
    "cerebral": {"Sci-Fi", "Mystery", "History"},
    "intense": {"Action", "War", "Horror"},
    "melancholic": {"Drama", "History"},
    "classic": {"Western", "Film-Noir"}
}

def get_mood_score(genres, preferred_moods):
    matched_moods = set()
    for g in genres:
        for mood, tags in mood_tone_map.items():
            if g['name'] in tags:
                matched_moods.add(mood)
    overlap = matched_moods & preferred_moods
    return len(overlap) / len(preferred_moods or [1])

def infer_moods_from_genres(genres):
    inferred = set()
    for g in genres:
        for mood, tags in mood_tone_map.items():
            if g in tags:
                inferred.add(mood)
    return inferred

def infer_mood_from_plot(plot):
    sentiment = sia.polarity_scores(plot)
    if sentiment['compound'] >= 0.4:
        return 'feel_good'
    elif sentiment['compound'] <= -0.3:
        return 'melancholic'
    else:
        return 'cerebral'

def compute_score(candidate, favorite_genres, favorite_actors, user_preferences):
    score = 0
    base_similarity_boost = 0.2
    score += base_similarity_boost

    # Genre match
    genre_match = len(set([g['name'] for g in candidate.genres]) & favorite_genres) / len(favorite_genres or [1])
    score += recommendation_weights["genre_similarity"] * genre_match

    # Cast/Crew match
    actor_match = len(set([a.name for a in list(candidate.cast)[:3]]) & favorite_actors) / len(favorite_actors or [1])
    score += recommendation_weights["cast_crew"] * actor_match

    # Year of Release
    year = int(candidate.release_date[:4]) if candidate.release_date else 0
    current_year = datetime.now().ye_


In [6]:
# Install necessary libraries
!pip install tmdbv3api nltk

# Import libraries
from tmdbv3api import TMDb, Movie, Person
from collections import defaultdict
from datetime import datetime
import concurrent.futures
import time
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

# TMDb setup
tmdb = TMDb()
tmdb.api_key = 'b2685c31a5f9519a6fa30472b7be2ca1'
tmdb.language = 'en'
tmdb.debug = True

movie = Movie()
person = Person()
sia = SentimentIntensityAnalyzer()

# Recommendation weights and streaming platform priorities
recommendation_weights = {
    "genre_similarity": 0.20,
    "cast_crew": 0.25,
    "release_year": 0.15,
    "ratings": 0.20,
    "streaming_availability": 0.10,
    "mood_tone": 0.05,
    "trending_factor": 0.05
}

streaming_platform_priority = {
    "netflix": 1.0,
    "disney_plus": 0.9,
    "hbo_max": 0.85,
    "hulu": 0.8,
    "prime_video": 0.75,
    "apple_tv": 0.7,
    "peacock": 0.6,
    "paramount_plus": 0.5,
}

mood_tone_map = {
    "feel_good": {"Comedy", "Romance", "Music", "Adventure"},
    "gritty": {"Crime", "Thriller", "Mystery", "Drama"},
    "cerebral": {"Sci-Fi", "Mystery", "History"},
    "intense": {"Action", "War", "Horror"},
    "melancholic": {"Drama", "History"},
    "classic": {"Western", "Film-Noir"}
}

def get_mood_score(genres, preferred_moods):
    matched_moods = set()
    for g in genres:
        for mood, tags in mood_tone_map.items():
            if g['name'] in tags:
                matched_moods.add(mood)
    overlap = matched_moods & preferred_moods
    return len(overlap) / len(preferred_moods or [1])

def infer_moods_from_genres(genres):
    inferred = set()
    for g in genres:
        for mood, tags in mood_tone_map.items():
            if g in tags:
                inferred.add(mood)
    return inferred

def infer_mood_from_plot(plot):
    sentiment = sia.polarity_scores(plot)
    if sentiment['compound'] >= 0.4:
        return 'feel_good'
    elif sentiment['compound'] <= -0.3:
        return 'melancholic'
    else:
        return 'cerebral'

def compute_score(candidate, favorite_genres, favorite_actors, user_preferences):
    score = 0
    base_similarity_boost = 0.2
    score += base_similarity_boost

    genre_match = len(set([g['name'] for g in candidate.genres]) & favorite_genres) / len(favorite_genres or [1])
    score += recommendation_weights["genre_similarity"] * genre_match

    actor_match = len(set([a.name for a in list(candidate.cast)[:3]]) & favorite_actors) / len(favorite_actors or [1])
    score += recommendation_weights["cast_crew"] * actor_match

    year = int(candidate.release_date[:4]) if candidate.release_date else 0
    current_year = datetime.now().year
    if current_year - year <= 2:
        score += recommendation_weights["release_year"] * 0.53
    elif current_year - year <= 5:
        score += recommendation_weights["release_year"] * 0.33
    elif current_year - year <= 15:
        score += recommendation_weights["release_year"] * 0.13

    score += recommendation_weights["ratings"] * (candidate.vote_average or 0) / 10

    platform = getattr(candidate, 'platform', None)
    if platform in user_preferences["subscribed_platforms"]:
        boost = streaming_platform_priority.get(platform, 0.5)
        score += recommendation_weights["streaming_availability"] * boost

    mood_score = get_mood_score(candidate.genres, user_preferences.get("preferred_moods", set()))
    score += recommendation_weights["mood_tone"] * mood_score

    return score

# Fetch similar movie details with caching
cache = {}
def fetch_similar_movie_details(m_id):
    if m_id in cache:
        return m_id, cache[m_id]
    try:
        m_details = movie.details(m_id)
        m_credits = movie.credits(m_id)
        m_details.genres = m_details.genres
        m_details.cast = list(m_credits['cast'])[:3]
        m_details.platform = None  # Placeholder
        m_details.plot = m_details.overview or ""
        cache[m_id] = m_details
        print(f"Fetched: {m_details.title}")
        return m_id, m_details
    except Exception as e:
        print(f"Failed to fetch movie ID {m_id}: {e}")
        return m_id, None

def recommend_movies(favorite_titles):
    start_time = time.time()
    favorite_genres = set()
    favorite_actors = set()
    candidate_movie_ids = set()
    inferred_plot_moods = set()

    for title in favorite_titles:
        search_result = movie.search(title)
        if not search_result:
            print(f"Movie '{title}' not found. Skipping.")
            continue

        movie_details = movie.details(search_result[0].id)
        movie_credits = movie.credits(search_result[0].id)

        favorite_genres.update([g['name'] for g in movie_details.genres])
        favorite_actors.update([c['name'] for c in list(movie_credits['cast'])[:3]])

        inferred_plot_moods.add(infer_mood_from_plot(movie_details.overview or ""))

        similar = movie.similar(movie_details.id)
        candidate_movie_ids.update([m.id for m in list(similar)])

    inferred_moods = infer_moods_from_genres(favorite_genres) | inferred_plot_moods
    user_preferences = {
        "subscribed_platforms": ["netflix", "hbo_max", "prime_video"],
        "preferred_moods": inferred_moods
    }

    candidate_movies = {}
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        future_to_id = {executor.submit(fetch_similar_movie_details, m_id): m_id for m_id in candidate_movie_ids}
        for future in concurrent.futures.as_completed(future_to_id):
            m_id, m = future.result()
            if m:
                candidate_movies[m_id] = m

    scored = [(m.title, compute_score(m, favorite_genres, favorite_actors, user_preferences)) for m in candidate_movies.values()]
    top_scored = sorted(scored, key=lambda x: x[1], reverse=True)[:10]

    print("\nTop 10 Recommendations Based on Your Favorites:")
    for title, score in top_scored:
        print(f"- {title} (score: {round(score, 2)})")

    print(f"\nTotal execution time: {round(time.time() - start_time, 2)} seconds")

# Example input
favorite_movies = [
    "The Bourne Identity",
    "Knocked Up",
    "Manchester by the Sea",
    "Gone Girl",
    "Miami Vice"
]

recommend_movies(favorite_movies)



[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Fetched: Kandahar
Fetched: Emekli Başkan
Fetched: Ala Vaikunthapurramuloo
Fetched: The Man from London
Fetched: Genome Hazard
Fetched: Kabul Express
Fetched: Sylvia
Fetched: Chloe
Fetched: La Vie de Bohème
Fetched: So Cold the River
Fetched: Rock Steady
Fetched: Birds Without Names
Fetched: Get a Clue
Fetched: The Secret Lives of Dentists
Fetched: The Nude Bomb
Fetched: The Stone Killer
Fetched: The Maltese Falcon
Fetched: Flags of Our Fathers
Fetched: Kamen Rider Ex-Aid the Movie: True Ending
Fetched: Attack of the Robots
Fetched: Lured
Fetched: Whisper Breach
Fetched: Rosenbaum - Bländverk
Fetched: Butch Cassidy and the Sundance Kid
Fetched: Last Train from Hiroshima
Fetched: Superhero Movie
Fetched: Rosenbaum - Målbrott
Fetched: The Tin Drum
Fetched: Cheese
Fetched: Orphan of Lowood
Fetched: Rosenbaum - Det sista vittnet
Fetched: The Cider House Rules
Fetched: Peut-être
Fetched: Promised a Miracle
Fetched: The Railway Station Man
Fetched: Tempting Fate
Fetched: Tower of London
Fetch

In [8]:
# Install necessary libraries
!pip install tmdbv3api nltk

# Import libraries
from tmdbv3api import TMDb, Movie, Person
from collections import defaultdict
from datetime import datetime
import concurrent.futures
import time
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

# TMDb setup
import locale

# Detect system locale for region customization
user_country = locale.getdefaultlocale()[0].split('_')[-1].lower()
tmdb = TMDb()
tmdb.api_key = 'b2685c31a5f9519a6fa30472b7be2ca1'
tmdb.language = 'en'
tmdb.region = user_country.upper()  # Adjust recommendations based on region
tmdb.debug = True

movie = Movie()
person = Person()
sia = SentimentIntensityAnalyzer()

# Recommendation weights and streaming platform priorities
recommendation_weights = {
    "genre_similarity": 0.20,
    "cast_crew": 0.25,
    "release_year": 0.15,
    "ratings": 0.20,
    "streaming_availability": 0.10,
    "mood_tone": 0.05,
    "trending_factor": 0.05
}

streaming_platform_priority = {
    "netflix": 1.0,
    "disney_plus": 0.9,
    "hbo_max": 0.85,
    "hulu": 0.8,
    "prime_video": 0.75,
    "apple_tv": 0.7,
    "peacock": 0.6,
    "paramount_plus": 0.5,
}

mood_tone_map = {
    "feel_good": {"Comedy", "Romance", "Music", "Adventure"},
    "gritty": {"Crime", "Thriller", "Mystery", "Drama"},
    "cerebral": {"Sci-Fi", "Mystery", "History"},
    "intense": {"Action", "War", "Horror"},
    "melancholic": {"Drama", "History"},
    "classic": {"Western", "Film-Noir"}
}

def get_mood_score(genres, preferred_moods):
    matched_moods = set()
    for g in genres:
        for mood, tags in mood_tone_map.items():
            if g['name'] in tags:
                matched_moods.add(mood)
    overlap = matched_moods & preferred_moods
    return len(overlap) / len(preferred_moods or [1])

def infer_moods_from_genres(genres):
    inferred = set()
    for g in genres:
        for mood, tags in mood_tone_map.items():
            if g in tags:
                inferred.add(mood)
    return inferred

def infer_mood_from_plot(plot):
    sentiment = sia.polarity_scores(plot)
    if sentiment['compound'] >= 0.4:
        return 'feel_good'
    elif sentiment['compound'] <= -0.3:
        return 'melancholic'
    else:
        return 'cerebral'

def compute_score(candidate, favorite_genres, favorite_actors, user_preferences):
    score = 0
    base_similarity_boost = 0.2
    score += base_similarity_boost

    genre_match = len(set([g['name'] for g in candidate.genres]) & favorite_genres) / len(favorite_genres or [1])
    score += recommendation_weights["genre_similarity"] * genre_match

    actor_match = len(set([a.name for a in list(candidate.cast)[:3]]) & favorite_actors) / len(favorite_actors or [1])
    score += recommendation_weights["cast_crew"] * actor_match

    year = int(candidate.release_date[:4]) if candidate.release_date else 0
    current_year = datetime.now().year
    if current_year - year <= 2:
        score += recommendation_weights["release_year"] * 0.53
    elif current_year - year <= 5:
        score += recommendation_weights["release_year"] * 0.33
    elif current_year - year <= 15:
        score += recommendation_weights["release_year"] * 0.13

    score += recommendation_weights["ratings"] * (candidate.vote_average or 0) / 10

    platform = getattr(candidate, 'platform', None)
    if platform in user_preferences["subscribed_platforms"]:
        boost = streaming_platform_priority.get(platform, 0.5)
        score += recommendation_weights["streaming_availability"] * boost

    mood_score = get_mood_score(candidate.genres, user_preferences.get("preferred_moods", set()))
    score += recommendation_weights["mood_tone"] * mood_score

    return score

# Fetch similar movie details with caching
cache = {}
def fetch_similar_movie_details(m_id):
    if m_id in cache:
        return m_id, cache[m_id]
    try:
        m_details = movie.details(m_id)
        m_credits = movie.credits(m_id)
        m_details.genres = m_details.genres
        m_details.cast = list(m_credits['cast'])[:3]
        m_details.platform = None  # Placeholder
        m_details.plot = m_details.overview or ""
        cache[m_id] = m_details
        print(f"Fetched: {m_details.title}")
        return m_id, m_details
    except Exception as e:
        print(f"Failed to fetch movie ID {m_id}: {e}")
        return m_id, None

def recommend_movies(favorite_titles):
    start_time = time.time()
    favorite_genres = set()
    favorite_actors = set()
    candidate_movie_ids = set()
    inferred_plot_moods = set()

    for title in favorite_titles:
        search_result = movie.search(title)
        if not search_result:
            print(f"Movie '{title}' not found. Skipping.")
            continue

        movie_details = movie.details(search_result[0].id)
        movie_credits = movie.credits(search_result[0].id)

        favorite_genres.update([g['name'] for g in movie_details.genres])
        favorite_actors.update([c['name'] for c in list(movie_credits['cast'])[:3]])

        inferred_plot_moods.add(infer_mood_from_plot(movie_details.overview or ""))

        similar = movie.similar(movie_details.id)
        candidate_movie_ids.update([m.id for m in list(similar)])

    inferred_moods = infer_moods_from_genres(favorite_genres) | inferred_plot_moods
        print(f"Using region: {user_country.upper()}")

    user_preferences = {
        "subscribed_platforms": ["netflix", "hbo_max", "prime_video"],
        "preferred_moods": inferred_moods
    }

    candidate_movies = {}
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        future_to_id = {executor.submit(fetch_similar_movie_details, m_id): m_id for m_id in candidate_movie_ids}
        for future in concurrent.futures.as_completed(future_to_id):
            m_id, m = future.result()
            if m:
                candidate_movies[m_id] = m

    scored = [(m.title, compute_score(m, favorite_genres, favorite_actors, user_preferences)) for m in candidate_movies.values()]
    top_scored = sorted(scored, key=lambda x: x[1], reverse=True)[:10]

    print("\nTop 10 Recommendations Based on Your Favorites:")
    for title, score in top_scored:
        print(f"- {title} (score: {round(score, 2)})")

    print(f"\nTotal execution time: {round(time.time() - start_time, 2)} seconds")

# Example input
favorite_movies = [
    "The Bourne Identity",
    "Knocked Up",
    "Manchester by the Sea",
    "Gone Girl",
    "Miami Vice"
]

recommend_movies(favorite_movies)


IndentationError: unexpected indent (<ipython-input-8-95bea9255c80>, line 162)

In [7]:
# Install necessary libraries
!pip install tmdbv3api nltk

# Import libraries
from tmdbv3api import TMDb, Movie, Person
from collections import defaultdict
from datetime import datetime
import concurrent.futures
import time
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

# TMDb setup
tmdb = TMDb()
tmdb.api_key = 'b2685c31a5f9519a6fa30472b7be2ca1'
tmdb.language = 'en'
tmdb.debug = True

movie = Movie()
person = Person()
sia = SentimentIntensityAnalyzer()

# Recommendation weights and streaming platform priorities
recommendation_weights = {
    "genre_similarity": 0.20,
    "cast_crew": 0.25,
    "release_year": 0.15,
    "ratings": 0.20,
    "streaming_availability": 0.10,
    "mood_tone": 0.05,
    "trending_factor": 0.05
}

streaming_platform_priority = {
    "netflix": 1.0,
    "disney_plus": 0.9,
    "hbo_max": 0.85,
    "hulu": 0.8,
    "prime_video": 0.75,
    "apple_tv": 0.7,
    "peacock": 0.6,
    "paramount_plus": 0.5,
}

mood_tone_map = {
    "feel_good": {"Comedy", "Romance", "Music", "Adventure"},
    "gritty": {"Crime", "Thriller", "Mystery", "Drama"},
    "cerebral": {"Sci-Fi", "Mystery", "History"},
    "intense": {"Action", "War", "Horror"},
    "melancholic": {"Drama", "History"},
    "classic": {"Western", "Film-Noir"}
}

def get_mood_score(genres, preferred_moods):
    matched_moods = set()
    for g in genres:
        for mood, tags in mood_tone_map.items():
            if g['name'] in tags:
                matched_moods.add(mood)
    overlap = matched_moods & preferred_moods
    return len(overlap) / len(preferred_moods or [1])

def infer_moods_from_genres(genres):
    inferred = set()
    for g in genres:
        for mood, tags in mood_tone_map.items():
            if g in tags:
                inferred.add(mood)
    return inferred

def infer_mood_from_plot(plot):
    sentiment = sia.polarity_scores(plot)
    if sentiment['compound'] >= 0.4:
        return 'feel_good'
    elif sentiment['compound'] <= -0.3:
        return 'melancholic'
    else:
        return 'cerebral'

def compute_score(candidate, favorite_genres, favorite_actors, user_preferences):
    score = 0
    base_similarity_boost = 0.2
    score += base_similarity_boost

    genre_match = len(set([g['name'] for g in candidate.genres]) & favorite_genres) / len(favorite_genres or [1])
    score += recommendation_weights["genre_similarity"] * genre_match

    actor_match = len(set([a.name for a in list(candidate.cast)[:3]]) & favorite_actors) / len(favorite_actors or [1])
    score += recommendation_weights["cast_crew"] * actor_match

    year = int(candidate.release_date[:4]) if candidate.release_date else 0
    current_year = datetime.now().year
    if current_year - year <= 2:
        score += recommendation_weights["release_year"] * 0.53
    elif current_year - year <= 5:
        score += recommendation_weights["release_year"] * 0.33
    elif current_year - year <= 15:
        score += recommendation_weights["release_year"] * 0.13

    score += recommendation_weights["ratings"] * (candidate.vote_average or 0) / 10

    platform = getattr(candidate, 'platform', None)
    if platform in user_preferences["subscribed_platforms"]:
        boost = streaming_platform_priority.get(platform, 0.5)
        score += recommendation_weights["streaming_availability"] * boost

    mood_score = get_mood_score(candidate.genres, user_preferences.get("preferred_moods", set()))
    score += recommendation_weights["mood_tone"] * mood_score

    return score

# Fetch similar movie details with caching
cache = {}
def fetch_similar_movie_details(m_id):
    if m_id in cache:
        return m_id, cache[m_id]
    try:
        m_details = movie.details(m_id)
        m_credits = movie.credits(m_id)
        m_details.genres = m_details.genres
        m_details.cast = list(m_credits['cast'])[:3]
        m_details.platform = None  # Placeholder
        m_details.plot = m_details.overview or ""
        cache[m_id] = m_details
        print(f"Fetched: {m_details.title}")
        return m_id, m_details
    except Exception as e:
        print(f"Failed to fetch movie ID {m_id}: {e}")
        return m_id, None

def recommend_movies(favorite_titles):
    print("
=== FAVORITE MOVIE MATCHING ===")
    start_time = time.time()
    favorite_genres = set()
    favorite_actors = set()
    candidate_movie_ids = set()
    inferred_plot_moods = set()

    for title in favorite_titles:
        search_result = movie.search(title)
        print(f"Searching: '{title}'")
        if not search_result:
            print(f"Movie '{title}' not found. Skipping.")
            continue

                movie_id = search_result[0].id
        print(f"Found: '{title}' → TMDb ID: {movie_id}")
        movie_details = movie.details(movie_id)
        movie_credits = movie.credits(search_result[0].id)

        favorite_genres.update([g['name'] for g in movie_details.genres])
        favorite_actors.update([c['name'] for c in list(movie_credits['cast'])[:3]])

        inferred_plot_moods.add(infer_mood_from_plot(movie_details.overview or ""))

                similar = movie.similar(movie_details.id)
        print(f"Similar movies for '{title}' → {[m.id for m in similar]}")
        candidate_movie_ids.update([m.id for m in list(similar)])

    inferred_moods = infer_moods_from_genres(favorite_genres) | inferred_plot_moods
    user_preferences = {
        "subscribed_platforms": ["netflix", "hbo_max", "prime_video"],
        "preferred_moods": inferred_moods
    }

    candidate_movies = {}
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        future_to_id = {executor.submit(fetch_similar_movie_details, m_id): m_id for m_id in candidate_movie_ids}
        for future in concurrent.futures.as_completed(future_to_id):
            m_id, m = future.result()
            if m:
                candidate_movies[m_id] = m

    scored = [(m.title, compute_score(m, favorite_genres, favorite_actors, user_preferences)) for m in candidate_movies.values()]
    top_scored = sorted(scored, key=lambda x: x[1], reverse=True)[:10]

    print("\nTop 10 Recommendations Based on Your Favorites:")
    for title, score in top_scored:
        print(f"- {title} (score: {round(score, 2)})")

    print(f"\nTotal execution time: {round(time.time() - start_time, 2)} seconds")

# Example input
favorite_movies = [
    "The Bourne Identity",
    "Knocked Up",
    "Manchester by the Sea",
    "Gone Girl",
    "Miami Vice"
]

recommend_movies(favorite_movies)


SyntaxError: unterminated string literal (detected at line 133) (<ipython-input-7-ec8b5b7a7dec>, line 133)

In [9]:
# Install necessary libraries
!pip install tmdbv3api nltk

# Import libraries
from tmdbv3api import TMDb, Movie, Person
from collections import defaultdict
from datetime import datetime
import concurrent.futures
import time
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

# TMDb setup
import locale

# Detect system locale for region customization
user_country = locale.getdefaultlocale()[0].split('_')[-1].lower()
tmdb = TMDb()
tmdb.api_key = 'b2685c31a5f9519a6fa30472b7be2ca1'
tmdb.language = 'en'
tmdb.region = user_country.upper()  # Adjust recommendations based on region
tmdb.debug = True

movie = Movie()
person = Person()
sia = SentimentIntensityAnalyzer()

# Recommendation weights and streaming platform priorities
recommendation_weights = {
    "genre_similarity": 0.20,
    "cast_crew": 0.25,
    "release_year": 0.15,
    "ratings": 0.20,
    "streaming_availability": 0.10,
    "mood_tone": 0.05,
    "trending_factor": 0.05
}

streaming_platform_priority = {
    "netflix": 1.0,
    "disney_plus": 0.9,
    "hbo_max": 0.85,
    "hulu": 0.8,
    "prime_video": 0.75,
    "apple_tv": 0.7,
    "peacock": 0.6,
    "paramount_plus": 0.5,
}

mood_tone_map = {
    "feel_good": {"Comedy", "Romance", "Music", "Adventure"},
    "gritty": {"Crime", "Thriller", "Mystery", "Drama"},
    "cerebral": {"Sci-Fi", "Mystery", "History"},
    "intense": {"Action", "War", "Horror"},
    "melancholic": {"Drama", "History"},
    "classic": {"Western", "Film-Noir"}
}

def get_mood_score(genres, preferred_moods):
    matched_moods = set()
    for g in genres:
        for mood, tags in mood_tone_map.items():
            if g['name'] in tags:
                matched_moods.add(mood)
    overlap = matched_moods & preferred_moods
    return len(overlap) / len(preferred_moods or [1])

def infer_moods_from_genres(genres):
    inferred = set()
    for g in genres:
        for mood, tags in mood_tone_map.items():
            if g in tags:
                inferred.add(mood)
    return inferred

def infer_mood_from_plot(plot):
    sentiment = sia.polarity_scores(plot)
    if sentiment['compound'] >= 0.4:
        return 'feel_good'
    elif sentiment['compound'] <= -0.3:
        return 'melancholic'
    else:
        return 'cerebral'

def compute_score(candidate, favorite_genres, favorite_actors, user_preferences):
    score = 0
    base_similarity_boost = 0.2
    score += base_similarity_boost

    genre_match = len(set([g['name'] for g in candidate.genres]) & favorite_genres) / len(favorite_genres or [1])
    score += recommendation_weights["genre_similarity"] * genre_match

    actor_match = len(set([a.name for a in list(candidate.cast)[:3]]) & favorite_actors) / len(favorite_actors or [1])
    score += recommendation_weights["cast_crew"] * actor_match

    year = int(candidate.release_date[:4]) if candidate.release_date else 0
    current_year = datetime.now().year
    if current_year - year <= 2:
        score += recommendation_weights["release_year"] * 0.53
    elif current_year - year <= 5:
        score += recommendation_weights["release_year"] * 0.33
    elif current_year - year <= 15:
        score += recommendation_weights["release_year"] * 0.13

    score += recommendation_weights["ratings"] * (candidate.vote_average or 0) / 10

    platform = getattr(candidate, 'platform', None)
    if platform in user_preferences["subscribed_platforms"]:
        boost = streaming_platform_priority.get(platform, 0.5)
        score += recommendation_weights["streaming_availability"] * boost

    mood_score = get_mood_score(candidate.genres, user_preferences.get("preferred_moods", set()))
    score += recommendation_weights["mood_tone"] * mood_score

    return score

# Fetch similar movie details with caching
cache = {}
def fetch_similar_movie_details(m_id):
    if m_id in cache:
        return m_id, cache[m_id]
    try:
        m_details = movie.details(m_id)
        m_credits = movie.credits(m_id)
        m_details.genres = m_details.genres
        m_details.cast = list(m_credits['cast'])[:3]
        m_details.platform = None  # Placeholder
        m_details.plot = m_details.overview or ""
        cache[m_id] = m_details
        print(f"Fetched: {m_details.title}")
        return m_id, m_details
    except Exception as e:
        print(f"Failed to fetch movie ID {m_id}: {e}")
        return m_id, None

def recommend_movies(favorite_titles):
    start_time = time.time()
    favorite_genres = set()
    favorite_actors = set()
    candidate_movie_ids = set()
    inferred_plot_moods = set()

    for title in favorite_titles:
        search_result = movie.search(title)
        if not search_result:
            print(f"Movie '{title}' not found. Skipping.")
            continue

        movie_details = movie.details(search_result[0].id)
        movie_credits = movie.credits(search_result[0].id)

        favorite_genres.update([g['name'] for g in movie_details.genres])
        favorite_actors.update([c['name'] for c in list(movie_credits['cast'])[:3]])

        inferred_plot_moods.add(infer_mood_from_plot(movie_details.overview or ""))

        similar = movie.similar(movie_details.id)
        candidate_movie_ids.update([m.id for m in list(similar)])

    inferred_moods = infer_moods_from_genres(favorite_genres) | inferred_plot_moods
        print(f"Using region: {user_country.upper()}")

    user_preferences = {
        "subscribed_platforms": ["netflix", "hbo_max", "prime_video"],
        "preferred_moods": inferred_moods
    }

    candidate_movies = {}
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        future_to_id = {executor.submit(fetch_similar_movie_details, m_id): m_id for m_id in candidate_movie_ids}
        for future in concurrent.futures.as_completed(future_to_id):
            m_id, m = future.result()
            if m:
                candidate_movies[m_id] = m

    scored = [(m.title, compute_score(m, favorite_genres, favorite_actors, user_preferences)) for m in candidate_movies.values()]
    top_scored = sorted(scored, key=lambda x: x[1], reverse=True)[:10]

    print("\nTop 10 Recommendations Based on Your Favorites:")
    for title, score in top_scored:
        print(f"- {title} (score: {round(score, 2)})")

    print(f"\nTotal execution time: {round(time.time() - start_time, 2)} seconds")

# Example input
favorite_movies = [
    "The Bourne Identity",
    "Knocked Up",
    "Manchester by the Sea",
    "Gone Girl",
    "Miami Vice"
]

recommend_movies(favorite_movies)


IndentationError: unexpected indent (<ipython-input-9-95bea9255c80>, line 162)

In [10]:
# Install necessary libraries
!pip install tmdbv3api nltk

# Import libraries
from tmdbv3api import TMDb, Movie, Person
from collections import defaultdict
from datetime import datetime
import concurrent.futures
import time
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

# TMDb setup
import locale

# Detect system locale for region customization
user_country = locale.getdefaultlocale()[0].split('_')[-1].lower()
tmdb = TMDb()
tmdb.api_key = 'b2685c31a5f9519a6fa30472b7be2ca1'
tmdb.language = 'en'
tmdb.region = user_country.upper()  # Adjust recommendations based on region
tmdb.debug = True

movie = Movie()
person = Person()
sia = SentimentIntensityAnalyzer()

# Recommendation weights and streaming platform priorities
recommendation_weights = {
    "genre_similarity": 0.20,
    "cast_crew": 0.25,
    "release_year": 0.15,
    "ratings": 0.20,
    "streaming_availability": 0.10,
    "mood_tone": 0.05,
    "trending_factor": 0.05
}

streaming_platform_priority = {
    "netflix": 1.0,
    "disney_plus": 0.9,
    "hbo_max": 0.85,
    "hulu": 0.8,
    "prime_video": 0.75,
    "apple_tv": 0.7,
    "peacock": 0.6,
    "paramount_plus": 0.5,
}

mood_tone_map = {
    "feel_good": {"Comedy", "Romance", "Music", "Adventure"},
    "gritty": {"Crime", "Thriller", "Mystery", "Drama"},
    "cerebral": {"Sci-Fi", "Mystery", "History"},
    "intense": {"Action", "War", "Horror"},
    "melancholic": {"Drama", "History"},
    "classic": {"Western", "Film-Noir"}
}

def get_mood_score(genres, preferred_moods):
    matched_moods = set()
    for g in genres:
        for mood, tags in mood_tone_map.items():
            if g['name'] in tags:
                matched_moods.add(mood)
    overlap = matched_moods & preferred_moods
    return len(overlap) / len(preferred_moods or [1])

def infer_moods_from_genres(genres):
    inferred = set()
    for g in genres:
        for mood, tags in mood_tone_map.items():
            if g in tags:
                inferred.add(mood)
    return inferred

def infer_mood_from_plot(plot):
    sentiment = sia.polarity_scores(plot)
    if sentiment['compound'] >= 0.4:
        return 'feel_good'
    elif sentiment['compound'] <= -0.3:
        return 'melancholic'
    else:
        return 'cerebral'

def compute_score(candidate, favorite_genres, favorite_actors, user_preferences):
    score = 0
    base_similarity_boost = 0.2
    score += base_similarity_boost

    genre_match = len(set([g['name'] for g in candidate.genres]) & favorite_genres) / len(favorite_genres or [1])
    score += recommendation_weights["genre_similarity"] * genre_match

    actor_match = len(set([a.name for a in list(candidate.cast)[:3]]) & favorite_actors) / len(favorite_actors or [1])
    score += recommendation_weights["cast_crew"] * actor_match

    year = int(candidate.release_date[:4]) if candidate.release_date else 0
    current_year = datetime.now().year
    if current_year - year <= 2:
        score += recommendation_weights["release_year"] * 0.53
    elif current_year - year <= 5:
        score += recommendation_weights["release_year"] * 0.33
    elif current_year - year <= 15:
        score += recommendation_weights["release_year"] * 0.13

    score += recommendation_weights["ratings"] * (candidate.vote_average or 0) / 10

    platform = getattr(candidate, 'platform', None)
    if platform in user_preferences["subscribed_platforms"]:
        boost = streaming_platform_priority.get(platform, 0.5)
        score += recommendation_weights["streaming_availability"] * boost

    mood_score = get_mood_score(candidate.genres, user_preferences.get("preferred_moods", set()))
    score += recommendation_weights["mood_tone"] * mood_score

    return score

# Fetch similar movie details with caching
cache = {}
def fetch_similar_movie_details(m_id):
    if m_id in cache:
        return m_id, cache[m_id]
    try:
        m_details = movie.details(m_id)
        m_credits = movie.credits(m_id)
        m_details.genres = m_details.genres
        m_details.cast = list(m_credits['cast'])[:3]
        m_details.platform = None  # Placeholder
        m_details.plot = m_details.overview or ""
        cache[m_id] = m_details
        print(f"Fetched: {m_details.title}")
        return m_id, m_details
    except Exception as e:
        print(f"Failed to fetch movie ID {m_id}: {e}")
        return m_id, None

def recommend_movies(favorite_titles):
    start_time = time.time()
    favorite_genres = set()
    favorite_actors = set()
    candidate_movie_ids = set()
    inferred_plot_moods = set()

    for title in favorite_titles:
        search_result = movie.search(title)
        if not search_result:
            print(f"Movie '{title}' not found. Skipping.")
            continue

        movie_details = movie.details(search_result[0].id)
        movie_credits = movie.credits(search_result[0].id)

        favorite_genres.update([g['name'] for g in movie_details.genres])
        favorite_actors.update([c['name'] for c in list(movie_credits['cast'])[:3]])

        inferred_plot_moods.add(infer_mood_from_plot(movie_details.overview or ""))

        similar = movie.similar(movie_details.id)
        candidate_movie_ids.update([m.id for m in list(similar)])

    inferred_moods = infer_moods_from_genres(favorite_genres) | inferred_plot_moods
    print(f"Using region: {user_country.upper()}")

    user_preferences = {
        "subscribed_platforms": ["netflix", "hbo_max", "prime_video"],
        "preferred_moods": inferred_moods
    }

    candidate_movies = {}
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        future_to_id = {executor.submit(fetch_similar_movie_details, m_id): m_id for m_id in candidate_movie_ids}
        for future in concurrent.futures.as_completed(future_to_id):
            m_id, m = future.result()
            if m:
                candidate_movies[m_id] = m

    scored = [(m.title, compute_score(m, favorite_genres, favorite_actors, user_preferences)) for m in candidate_movies.values()]
    top_scored = sorted(scored, key=lambda x: x[1], reverse=True)[:10]

    print("\nTop 10 Recommendations Based on Your Favorites:")
    for title, score in top_scored:
        print(f"- {title} (score: {round(score, 2)})")

    print(f"\nTotal execution time: {round(time.time() - start_time, 2)} seconds")

# Example input
favorite_movies = [
    "The Bourne Identity",
    "Knocked Up",
    "Manchester by the Sea",
    "Gone Girl",
    "Miami Vice"
]

recommend_movies(favorite_movies)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
<ipython-input-10-130ecd16614e>:18: DeprecationWarning: 'locale.getdefaultlocale' is deprecated and slated for removal in Python 3.15. Use setlocale(), getencoding() and getlocale() instead.
  user_country = locale.getdefaultlocale()[0].split('_')[-1].lower()


Using region: US
Fetched: Kandahar
Fetched: Emekli Başkan
Fetched: Ala Vaikunthapurramuloo
Fetched: The Man from London
Fetched: Genome Hazard
Fetched: Kabul Express
Fetched: Sylvia
Fetched: Chloe
Fetched: La Vie de Bohème
Fetched: So Cold the River
Fetched: Rock Steady
Fetched: Birds Without Names
Fetched: Get a Clue
Fetched: The Secret Lives of Dentists
Fetched: The Nude Bomb
Fetched: The Stone Killer
Fetched: The Maltese Falcon
Fetched: Flags of Our Fathers
Fetched: Kamen Rider Ex-Aid the Movie: True Ending
Fetched: Attack of the Robots
Fetched: Whisper Breach
Fetched: Rosenbaum - Bländverk
Fetched: Butch Cassidy and the Sundance Kid
Fetched: Last Train from Hiroshima
Fetched: Superhero Movie
Fetched: Rosenbaum - Målbrott
Fetched: The Tin Drum
Fetched: Cheese
Fetched: Orphan of Lowood
Fetched: Rosenbaum - Det sista vittnet
Fetched: The Cider House Rules
Fetched: Peut-être
Fetched: Promised a Miracle
Fetched: Lured
Fetched: The Railway Station Man
Fetched: Tempting Fate
Fetched: Towe